<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Meta_Llama_3_8B_for_MEDAL_EVALUATOR_evaldata_NEW_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sudo apt install nvtop

# Dependencies

In [1]:
!ps aux |grep python3 |grep colab_kernel

root         748  8.6  0.1 1510352 97236 ?       Ssl  18:12   0:02 /usr/bin/python3 -m colab_kernel_


In [2]:
#!pip install --upgrade huggingface_hub -q

In [3]:
#!pip install huggingface_hub -q

https://medium.com/@frankmorales_91352/fine-tuning-meta-llama-3-8b-with-medal-for-enhanced-medical-language-understanding-3de07a54fab9

In [ ]:
! pip install peft --quiet

!pip install -U flash-attn --no-build-isolation --quiet

#Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

In [5]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [ ]:
!pip install colab-env --quiet

# Hugging Face Setup

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [8]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Tensorboard Setup

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
#!git clone https://huggingface.co/frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2
# https://huggingface.co/frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata
!git clone https://huggingface.co/frankmorales2020/POC-NEW-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata

In [ ]:
%load_ext tensorboard
#%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B-MEDAL-flash-attention-2/runs
#%tensorboard --logdir /content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2/runs/

#%tensorboard --logdir /content/tensorbard/POC-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata/logs
%tensorboard --logdir /content/tensorbard/POC-NEW-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata/logs
#%tensorboard --logdir /content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine/runs/


# Dataset Load

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

# Load Model

In [12]:
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2"
#peft_model_id = 'frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2'

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine"
#peft_model_id = 'frankmorales2020/POC-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata'
peft_model_id = 'frankmorales2020/POC-NEW-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata'


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

#model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',  load_in_4bit=True, ...)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [14]:
torch.cuda.empty_cache()

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
#/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
#nrec= randint(0, len(eval_dataset))

nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [16]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'[/INST] antral follicle count </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antihypertensive agent to the treatment of patients with resistant hypertension who were receiving a combination of two or three antihypertensive agents the T0 was a prospective randomized doubleblind placebocontrolled trial of patients with resistant hypertension who were receiving a combination of two or three antihypertensive agents the patients were randomized to receive either the new agent or placebo for weeks the primary outcome was the change in systolic blood pressure sbp from baseline to week the AA outcomes were'

In [17]:
def similarity(ganswer,oanswer):

  qc0=str(ganswer).find('[INST]')

  ganswer=str(ganswer)[0:qc0-8]
  qc=str(ganswer).find('[/INST]')
  if qc>0:
    ganswer=ganswer[qc+8:len(ganswer)]

  print(f"Generated Answer:\n{ganswer}")

  if ganswer == oanswer:
    print("Match")
    match=1
  else:
    print("NO Match")
    match=0
  return match

In [18]:
print(f"Query:\n{eval_dataset[nrec]['text']}")

oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")

ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

match=similarity(ganswer,oanswer)

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

# Model Evaluation - Kernel

In [19]:
torch.cuda.empty_cache()

In [20]:
!pip install evaluate -q

In [21]:
# Load and Prepare Dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
reduced_size = 100
eval_dataset = eval_dataset.shuffle(seed=42).select(range(reduced_size))

In [22]:
max_length = 10
all_input_ids = []
all_attention_masks = []
all_labels = []

for text, label in zip(eval_dataset["text"], eval_dataset["label"]):
    # Flatten nested lists if necessary
    text = " ".join(text) if isinstance(text, list) else text
    label = " ".join(label) if isinstance(label, list) else label
    # Tokenize input
    tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    all_input_ids.append(tokenized["input_ids"][0])
    all_attention_masks.append(tokenized["attention_mask"][0])
    # Encode the labels using the tokenizer
    # This is essential for tasks like text classification
    label_id = tokenizer.encode(label, add_special_tokens=False)
    all_labels.append(torch.tensor(label_id))


input_ids = torch.stack(all_input_ids)
attention_masks = torch.stack(all_attention_masks)
labels = torch.nn.utils.rnn.pad_sequence(all_labels, batch_first=True, padding_value=-100)

# Truncate the labels
labels = labels[:, :max_length]  # Truncate to the first max_length tokens

In [23]:
# Now you have input_ids, attention_masks, and labels as tensors with compatible shapes
print(input_ids.shape)
print(attention_masks.shape)
print(labels.shape)

torch.Size([100, 10])
torch.Size([100, 10])
torch.Size([100, 10])


In [24]:
# Count Hidden Layers and Neurons (Before Evaluation)
if hasattr(model, 'base_model'):
    llama_model = model.base_model
else:
    llama_model = model

# Count hidden layers of type LlamaDecoderLayer
num_hidden_layers = llama_model.config.num_hidden_layers
#print(num_hidden_layers)

# Estimate neurons (this is very simplified, as explained earlier)
num_neurons = num_hidden_layers * llama_model.config.hidden_size

print(f"Number of hidden layers in the model: {num_hidden_layers}")
print(f"Approximate number of neurons (simplified): {num_neurons}")


Number of hidden layers in the model: 32
Approximate number of neurons (simplified): 131072


In [25]:
torch.cuda.empty_cache()

In [27]:
!nvidia-smi

Mon Jun 24 18:28:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |  39343MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM
import evaluate
from tqdm import tqdm

# Constants
BATCH_SIZE = 1   # Batch size is set to 1
MAX_LENGTH = 10
peft_model_id = 'frankmorales2020/POC-NEW-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata'

# Load and Prepare Dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
reduced_size = 5
eval_dataset = eval_dataset.shuffle(seed=42).select(range(reduced_size))

# Tokenization and Tensor Creation
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])
tokenized_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Create DataLoader (directly from dataset)
eval_dataloader = DataLoader(tokenized_eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Move model to GPU (if available) - Assuming you have already done this
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)
device='cuda'

# Evaluation Function (with progress bar)
metric = evaluate.load("perplexity")

#metric = evaluate.load("accuracy")

def evaluate_model(model, eval_dataloader, metric, model_id):
    model.eval()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch, labels=batch["input_ids"])

        logits = outputs.logits
        predictions = logits.argmax(dim=-1)
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)
        metric.add_batch(predictions=decoded_predictions, references=decoded_labels)

    return metric.compute(model_id=model_id)

# Perform Evaluation
results = evaluate_model(model, eval_dataloader, metric, peft_model_id)

In [28]:
print(results)

{'perplexities': [1415.568115234375, 2708.177734375, 1152.5687255859375, 1820.6805419921875, 339.6514587402344], 'mean_perplexity': 1487.3293151855469}


# Model Evaluation - Inference

In [29]:
torch.cuda.empty_cache()

In [30]:
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [31]:
eval_dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [32]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [33]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
else:
    ganswer=ganswer[qc+8:len(ganswer)]
ganswer

'antral follicle count </s><s>[INST] the aim of this study was to evaluate the effect of the addition of a new antihypertensive agent to the treatment of patients with resistant hypertension who were receiving a combination of two or three antihypertensive agents the T0 was a prospective randomized doubleblind placebocontrolled trial of patients with resistant hypertension who were receiving a combination of two or three antihypertensive agents the patients were randomized to receive either the new agent or placebo for weeks the primary outcome was the change in systolic blood pressure sbp from baseline to week the AA outcomes were'

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

In [35]:
def exceptions(ganswer,cganswer,eganswer):
     if ganswer==eganswer:
        ganswer=cganswer
     return ganswer

In [36]:
# ... (your other imports and model setup)

# Assuming you've already split your data into training and testing

#train_dataset = ...  #  3M training samples
#test_dataset = ...   #  1M testing samples

# Sample randomly to create a validation set from the test set
#eval_dataset = test_dataset.select(range(10807))  # Or use a more sophisticated sampling method

# ... (rest of your SFTTrainer code)

# ... (In your TrainingArguments, set `load_best_model_at_end=True`)

# After training, evaluate on the full test set
#trainer.evaluate(eval_dataset=test_dataset)


In [37]:
torch.cuda.empty_cache()

In [38]:
from tqdm import tqdm


def evaluate(sample):
    prompt =  sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer=outputs[0]['generated_text'][len(prompt):].strip()

    #print()
    #print(f"Question:\n{sample['text']}")
    #print()

    #print()
    #print(f"Generated Answer original:\n{ganswer}")
    #print()

    # molecule [/INST] phosphorylethanolamine </s><s>[INST]
    qc0=str(ganswer).find('[/INST]')
    qc1=str(ganswer).find('[INST]')

    #print(qc0)
    #print(qc1)

    ganswer=str(ganswer)[qc0+8:qc1-8]
    #print(f"Generated Answer corr:\n{ganswer}")

    oanswer=str(sample['label'])
    oanswer=oanswer[2:len(oanswer)-2]


    #### exceptions TODO

    #if ganswer == 'enteropathogenic':
    #   ganswer=exceptions(ganswer,oanswer,ganswer)


    #print(len(oanswer))
    #print(len(ganswer))

    #print(f"Original Answer:\n{oanswer}")
    if ganswer == oanswer:
        print()
        print()
        print("Match")
        print(f"Query:\n{sample['text']}")
        print(f"Original Answer:\n{oanswer}")
        print(f"Generated Answer:\n{ganswer}")
        print()
        return 1
    else:
        print()
        print()
        print("no Match")
        print(f"Query:\n{sample['text']}")
        print(f"Original Answer:\n{oanswer}")
        print(f"Generated Answer:\n{ganswer}")
        print()
        return 0


#### exceptions TODO

#Original Answer:
#enteropathogenic
#Generated Answer:
#enteropathogenic e coli

#Original Answer:
#severe mental retardation
#Generated Answer:
#mental retardation

#Original Answer:
#eye movement
#Generated Answer:
#eye movements


#Original Answer:
#severe mental retardation
#Generated Answer:
#mental retardation


success_rate = []
number_of_eval_samples = 5
# iterate over eval dataset and predict

#for n in tqdm(range(number_of_eval_samples)):
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

 20%|██        | 1/5 [00:18<01:13, 18.29s/it]



no Match
Query:
we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in c in bal total cells were x ml in a in b and in c macrophages in bal were in a in b and in c neutrophils were in a in b in c eosinophils were in a in b and in c p less than group c versus group a total proteins albumin ALP phosphatase and fibronectin

 40%|████      | 2/5 [00:36<00:54, 18.20s/it]



Match
Query:
pyogenic granulomas represent the aquisition of vasodilative granulation tissue in the skin or mucosa they are extremely rare in the alimentary tract other than in the oral cavity here we report a case of PG arising from the GM an yearold man was admitted to our hospital because of melena of more than months duration esophagogastroduodenoscopy egd revealed a mmdiameter semipedunculated lesion with an irregular surface in the fundus of the stomach during hospitalization the patients anemia worsened due to loss of blood from the lesion with the level of hemoglobin declining to gdl and a blood transfusion was required because radiological and endoscopic findings indicated the lesion was hypervascular transarterial embolization of the nutritional i.a. of the lesion was performed before ER of the lesion one week T3 the embolotherapy endoscopic mucosal resection was performed without any complications such as massive one histological studies of the resected specimen revealed m

 60%|██████    | 3/5 [00:54<00:36, 18.20s/it]



no Match
Query:
the l immunotype lipopolysaccharide lps of neisseria meningitidis was subjected to Kd procedures which produced a number of different oligosaccharide fragments the high resolution h and c nmr spectroscopic analyses of these OS yielded structural information on a number of different regions of the lps for example from one oligosaccharide it was found that the endogenous sialylation of the meningococcal lps occurs at o of the terminal betadgalactopyranosyl residue of its lactonneotetraose antenna in the alphadconfiguration from another it was also established that the dominant structural feature responsible for l epitope specificity is the presence of a PE substituent at o of the penultimate heptopyranosyl residue of its other antenna in addition from information obtained with another oligosaccharide the structure of the deoxydmannooctulosonic acid disaccharide region of the l lps was also elucidated from all the above cumulative data plus some published data it was the

 80%|████████  | 4/5 [01:12<00:18, 18.17s/it]



Match
Query:
inotropic reserve identified by dobutamine or dipyridamole SE is associated with a better outcome in patients with idiopathic dilated cardiomyopathy dcm although the relative prognostic value of each remains unsettled the purpose of the present study was to assess the relative prognostic value of dobutamine versus dipyridamole SE for the prediction of allcause death in patients with idiopathic dcm
Original Answer:
stress echocardiography
Generated Answer:
stress echocardiography



100%|██████████| 5/5 [01:30<00:00, 18.18s/it]



Match
Query:
pyridoxinedependent seizure is a rare autosomal recessive disorder that usually presents with neonatal intractable seizures this syndrome results from an inborn abnormality of the enzyme glu decarboxylase which results in reduced pyridazinedependent synthesis of the inhibitory neurotransmitter gamma amino butyric acid the full range of symptomatology is unknown but can be associated with autism breath holding and SMR bilious vomiting transient visual agnosia severe articulatory apraxia motor dyspraxia microcephaly and intrauterine seizures parenteral pyridine injection test is a highly ERP and reproducible test in confirming the diagnosis pyridoxine should be po as a diagnostic test in all cases of convulsive disorders of infancy in which no other diagnosis is evident epileptic seizure discharges subside within minutes T3 the i.v. of mg of pyridaoxine once the diagnosis is confirmed MT should be continued indefinitely and doses increased with age or intercurrent illnesse

In [39]:
torch.cuda.empty_cache()

In [40]:
#print(f"Accuracy: {accuracy*100:.2f}%")

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 5: 60.00%



learning_rate: 2e-05
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 6
total_train_batch_size: 18
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: constant
lr_scheduler_warmup_ratio: 0.03
num_epochs: 2

When evaluated on 1000 samples from the evaluation dataset, our model achieved an accuracy of 26.30%. However, there's room for improvement.


learning_rate: 0.0002
train_batch_size: 3
eval_batch_size: 8
seed: 42
gradient_accumulation_steps: 6
total_train_batch_size: 18
optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
lr_scheduler_type: cosine
lr_scheduler_warmup_ratio: 0.03
num_epochs: 10

When evaluated on 10 samples from the evaluation dataset, our model achieved an accuracy of 60%. However, there's room for improvement.

